# Langchaing Conversation Model
The following code allows us to use conversation models with langchain


In [62]:
import os
import openai
import pandas as pd
    
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [70]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [71]:
data = pd.read_json('info.json')
data.head(5)

,product,review
0,SuperTech Blender,This blender is amazing! It blends everything ...
1,EcoClean Dishwasher,I'm quite disappointed with this dishwasher. I...
2,GourmetChef Knife Set,I absolutely love this knife set. The blades a...
3,CozyDreams Weighted Blanket,This weighted blanket is so comforting. It hel...
4,TechGizmo Smart Watch,"The smartwatch has a lot of potential, but the..."


In [72]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [73]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [69]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [75]:
chain = LLMChain(llm=llm, prompt=prompt)

In [76]:
# Proving the chain
product = "Queen size bed"
chain.run(product)

'Regal Rest'

In [77]:
from langchain.chains import SimpleSequentialChain

# Prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [78]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [80]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [81]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Slumber
Royal Slumber is a luxury bedding company that offers premium, comfortable, and stylish sleep essentials fit for royalty.

> Finished chain.


'Royal Slumber is a luxury bedding company that offers premium, comfortable, and stylish sleep essentials fit for royalty.'

In [85]:
# Sequential chains are for multiple inputs
from langchain.chains import SequentialChain

# prompt template 1: translate to Spanish
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to spanish:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= Spanish_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="Spanish_Review"
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you give me the product name and the felling [good, bad] of the comment:"
    "\n\n{Spanish_Review}"
)
# chain 2: input= Spanish_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" # Using output_key is important
                    )

overall_chain = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=["Review"],
    output_variables=["Spanish_Review", "summary"],
    verbose=True
)                   

In [86]:
review = data.review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "These resistance bands are perfect for my home workouts. They're durable and versatile.",
 'Spanish_Review': 'Estas bandas de resistencia son perfectas para mis entrenamientos en casa. Son duraderas y versátiles.',
 'summary': 'Product name: Bandas de resistencia \nFeeling: Good'}

In [101]:
# Router Chain
good_template = """You are a customer service agent \
You are happy that the review is good, so you going to congrats\
this customer for buying it  \

Here is the review:
{input}"""


bad_template = """You are a customer service agent \
You are sad that the review is bad, so you want to encourage\
the client to contact sales team for further support  \

Here is a review:
{input}"""

prompt_infos = [
    {
        "name": "good", 
        "description": "Good for good reviews", 
        "prompt_template": good_template
    },
    {
        "name": "math", 
        "description": "Good for bad reviews",  
        "prompt_template": bad_template
    },
]

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser # Route between subchains
from langchain.prompts import PromptTemplate

In [102]:
llm = ChatOpenAI(temperature=0, model=llm_model)
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [103]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [104]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [105]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)  

In [106]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [109]:
chain.run(data.review[7])



> Entering new MultiPromptChain chain...


C:\Anaconda3\lib\site-packages\langchain\chains\llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': "This pan's non-stick coating started peeling after just a few uses. Very poor quality."}
> Finished chain.


'Dear valued customer,\n\nI am sorry to hear that you had a negative experience with our product. We take all feedback seriously and strive to provide the best quality products to our customers.\n\nI would like to encourage you to contact our sales team for further support. They will be able to assist you with any issues you may have and provide you with a solution that meets your needs.\n\nThank you for taking the time to share your feedback with us. We appreciate your business and hope to have the opportunity to serve you better in the future.\n\nBest regards,\n\n[Your Name]'